<a href="https://colab.research.google.com/github/skatmddhks/Instagram-Like-prediction/blob/main/%E1%84%8B%E1%85%B5%E1%86%AB%E1%84%89%E1%85%B3%E1%84%90%E1%85%A1%E1%84%8C%E1%85%A9%E1%87%82%E1%84%8B%E1%85%A1%E1%84%8B%E1%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#인스타 좋아요 예측하기


*   결과: 이미지 vgg16, 텍스트 lstm & attention을 병합했을때 가장 결과가 좋았으나 좋아요 예측모델에 대한 변수가 부적합 했다.



In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout, Embedding, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
import re
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 데이터 로드
data = pd.read_csv('/content/all_data.csv')  # 데이터 파일 경로를 적절히 수정해주세요.




In [ ]:
data

,Content,Hashtags,Likes,Follower
0,@poloralphlauren💙🩵,"#OnlyPolo, #PoloRalphLauren, #폴로랄프로렌",90.9만개,9500000
1,막방까지👽❤️,NaN,84.8만개,9500000
2,고양이놀이😽🐱,NaN,90.6만개,9500000
3,🩵아카라카💙,NaN,80만개,9500000
4,🎀,NaN,94만개,9500000
...,...,...,...,...
8176,✨,#highcut,200.5만개,78470000
8177,요즘날씨짱🤩🤩🤩🤩🤩🤩🤩🤩🌸,"#광고, #ADIDAS, #JISOO",193.6만개,78470000
8178,🌸haveagoodday🌸,NaN,201.7만개,78470000
8179,엄청큰지수와핑크단발지수의만남💕둘다지수인데왜큰지수만보는거죠😭😭,"#BLACKPINK, #JISOO",217.6만개,78470000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8181 entries, 0 to 8180
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Content   6916 non-null   object
 1   Hashtags  1456 non-null   object
 2   Likes     7819 non-null   object
 3   Follower  8181 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 255.8+ KB


In [ ]:
# 좋아합니다를 포함한 행의 개수 확인
likes_contains_good = data[data['Likes'].str.contains('좋아합니다', na=False)]
likes_contains_good_count = len(likes_contains_good)

print("좋아합니다를 포함한 행의 개수:", likes_contains_good_count)

좋아합니다를 포함한 행의 개수: 234


In [ ]:
likes_contains_good

,Content,Hashtags,Likes,Follower
383,수수수수수퍼노바첫째주끝..힘들었다야🦭🦭..,NaN,youngji_02\n님 외 136.1만명이 좋아합니다,7250000
384,음방하면서깜빡해서사진못찍었어…미아내앵,NaN,youngji_02\n님 외 123.6만명이 좋아합니다,7250000
392,자연사랑투어리스트닝🦛,NaN,youngji_02\n님 외 161.6만명이 좋아합니다,7250000
394,KconHongKong2024후~엠씨핫데뷔했다ㅎㅎㅎ,NaN,youngji_02\n님 외 121.2만명이 좋아합니다,7250000
395,@versacein@BEIJINGSLT🩷下次再见北京,NaN,youngji_02\n님 외 124.1만명이 좋아합니다,7250000
...,...,...,...,...
2629,UmpahUmpahisoutnowlinkinbio💋,NaN,yerin_the_genuine\n님 외 47.7만명이 좋아합니다,10770000
2632,Thefestivalwillcontinue,NaN,yerin_the_genuine\n님 외 45.3만명이 좋아합니다,10770000
2654,😜,NaN,yerin_the_genuine\n님 외 45.3만명이 좋아합니다,10770000
2656,히히히,NaN,yerin_the_genuine\n님 외 54.8만명이 좋아합니다,10770000


In [ ]:
def preprocess_likes(likes):
    if pd.notnull(likes):
        if '좋아합니다' in likes:
            match = re.search(r'외\s+(\d+(\.\d+)?)만', likes)
            if match:
                return float(match.group(1)) * 10000
            else:
                return np.nan  # '외' 이후에 숫자가 없는 경우 NaN 반환
        elif '만개' in likes:
            return float(likes.replace('만개', '')) * 10000
        else:
            # '좋아합니다'도 '만개'도 없는 경우
            match = re.search(r'외\s+(\d+(\.\d+)?)만', likes)
            if match:
                return float(match.group(1))
            else:
                return np.nan
    else:
        return np.nan  # 결측치인 경우 NaN 반환

# Likes 열에 대해 preprocess_likes 함수를 적용합니다.
data['Likes'] = data['Likes'].apply(preprocess_likes)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8181 entries, 0 to 8180
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Content   6916 non-null   object 
 1   Hashtags  1456 non-null   object 
 2   Likes     7798 non-null   float64
 3   Follower  8181 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 255.8+ KB


In [ ]:
data

,Content,Hashtags,Likes,Follower
0,@poloralphlauren💙🩵,"#OnlyPolo, #PoloRalphLauren, #폴로랄프로렌",909000.0,9500000
1,막방까지👽❤️,NaN,848000.0,9500000
2,고양이놀이😽🐱,NaN,906000.0,9500000
3,🩵아카라카💙,NaN,800000.0,9500000
4,🎀,NaN,940000.0,9500000
...,...,...,...,...
8176,✨,#highcut,2005000.0,78470000
8177,요즘날씨짱🤩🤩🤩🤩🤩🤩🤩🤩🌸,"#광고, #ADIDAS, #JISOO",1936000.0,78470000
8178,🌸haveagoodday🌸,NaN,2017000.0,78470000
8179,엄청큰지수와핑크단발지수의만남💕둘다지수인데왜큰지수만보는거죠😭😭,"#BLACKPINK, #JISOO",2176000.0,78470000


In [ ]:
# 결측치를 공백으로 채웁니다.
data['Content'].fillna('', inplace=True)
data['Hashtags'].fillna('', inplace=True)

In [ ]:
# Likes 열의 결측치를 앞의 값으로 채웁니다.
data['Likes'].fillna(method='ffill', inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8181 entries, 0 to 8180
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Content   8181 non-null   object 
 1   Hashtags  8181 non-null   object 
 2   Likes     8181 non-null   float64
 3   Follower  8181 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 255.8+ KB


In [ ]:
# 'Likes'와 'Follower' 컬럼의 데이터 타입 변환 (문자열 -> 숫자)
data['Likes'] = pd.to_numeric(data['Likes'], errors='coerce')
data['Follower'] = pd.to_numeric(data['Follower'], errors='coerce')

# Engagement Rate 계산
data['Engagement_Rate'] = data['Likes'] / data['Follower']

data

,Content,Hashtags,Likes,Follower,Engagement_Rate
0,@poloralphlauren💙🩵,"#OnlyPolo, #PoloRalphLauren, #폴로랄프로렌",909000.0,9500000,0.095684
1,막방까지👽❤️,,848000.0,9500000,0.089263
2,고양이놀이😽🐱,,906000.0,9500000,0.095368
3,🩵아카라카💙,,800000.0,9500000,0.084211
4,🎀,,940000.0,9500000,0.098947
...,...,...,...,...,...
8176,✨,#highcut,2005000.0,78470000,0.025551
8177,요즘날씨짱🤩🤩🤩🤩🤩🤩🤩🤩🌸,"#광고, #ADIDAS, #JISOO",1936000.0,78470000,0.024672
8178,🌸haveagoodday🌸,,2017000.0,78470000,0.025704
8179,엄청큰지수와핑크단발지수의만남💕둘다지수인데왜큰지수만보는거죠😭😭,"#BLACKPINK, #JISOO",2176000.0,78470000,0.027730


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8181 entries, 0 to 8180
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Content          8181 non-null   object 
 1   Hashtags         8181 non-null   object 
 2   Likes            8181 non-null   float64
 3   Follower         8181 non-null   int64  
 4   Engagement_Rate  8181 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 319.7+ KB


In [ ]:
data.to_csv('결측치채운버전.csv')

## 팔로워, 참여도 추가 버전 (해시태그, 게시글 따로따로, 팔로워, 비율 따로따로)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# 이미지 데이터 전처리 함수
def preprocess_images(folders):
    all_image_data = []
    total_image_count = 0
    for folder in folders:
        image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder) if filename.endswith(('.png', '.jpg', '.jpeg'))]
        total_image_count += len(image_paths)  # 폴더 내 이미지 파일 수를 누적합니다.
        for img_path in image_paths:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            all_image_data.append(img_array)
    print(f"총 이미지 파일 개수: {total_image_count}")
    return np.vstack(all_image_data)

# 이미지 폴더 목록
folders = [
    '/content/drive/MyDrive/공유폴덩/에스파_윈터',
    '/content/drive/MyDrive/공유폴덩/에스파_카리나',
    '/content/drive/MyDrive/공유폴덩/에스파_지젤',
    '/content/drive/MyDrive/공유폴덩/에스파_닝닝',
    '/content/drive/MyDrive/공유폴덩/레드벨벳_아이린',
    '/content/drive/MyDrive/공유폴덩/레드벨벳_슬기',
    '/content/drive/MyDrive/공유폴덩/레드벨벳_웬디',
    '/content/drive/MyDrive/공유폴덩/레드벨벳_예리',
    '/content/drive/MyDrive/공유폴덩/레드벨벳_조이',
    '/content/drive/MyDrive/공유폴덩/블랙핑크_제니',
    '/content/drive/MyDrive/공유폴덩/블랙핑크_로제',
    '/content/drive/MyDrive/공유폴덩/블랙핑크_리사',
    '/content/drive/MyDrive/공유폴덩/블랙핑크_지수'
]

# 이미지 데이터를 전처리하고 이미지 개수를 출력합니다.
all_image_data = preprocess_images(folders)


총 이미지 파일 개수: 8181


In [ ]:
# 텍스트 데이터 임베딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Content'].tolist() + data['Hashtags'].tolist())
max_seq_length = max([len(seq) for seq in tokenizer.texts_to_sequences(data['Content'].tolist() + data['Hashtags'].tolist())])
vocab_size = len(tokenizer.word_index) + 1

def preprocess_text(texts, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

X_content = preprocess_text(data['Content'], tokenizer, max_seq_length)  # 게시글
X_hashtags = preprocess_text(data['Hashtags'], tokenizer, max_seq_length) # 해시태그

# 데이터 준비
X_image = all_image_data  #이미지 변수
y = data['Likes'].values
X_follower = data['Follower'].values  #팔로워
X_engagement = data['Engagement_Rate'].values  #비율

# 데이터 분할
X_content_train, X_content_val, X_hashtags_train, X_hashtags_val, X_image_train, X_image_val, y_train, y_val, X_follower_train, X_follower_val, X_engagement_train, X_engagement_val = train_test_split(
    X_content, X_hashtags, X_image, y, X_follower, X_engagement, test_size=0.2, random_state=42)

# 이미지 입력 모델
image_model = Sequential()
image_model.add(Input(shape=(224, 224, 3)))
image_model.add(VGG16(weights='imagenet', include_top=False))
image_model.add(GlobalAveragePooling2D())
image_model.add(Dense(512, activation='relu'))
image_model.add(Dropout(0.5))

# 텍스트 입력 모델 (Content)
content_model = Sequential()
content_model.add(Input(shape=(max_seq_length,)))
content_model.add(Embedding(vocab_size, 100, input_length=max_seq_length))
content_model.add(Flatten())
content_model.add(Dense(512, activation='relu'))
content_model.add(Dropout(0.5))

# 텍스트 입력 모델 (Hashtags)
hashtags_model = Sequential()
hashtags_model.add(Input(shape=(max_seq_length,)))
hashtags_model.add(Embedding(vocab_size, 100, input_length=max_seq_length))
hashtags_model.add(Flatten())
hashtags_model.add(Dense(512, activation='relu'))
hashtags_model.add(Dropout(0.5))

# 추가 입력 모델 (Follower, Engagement Rate)
follower_input = Input(shape=(1,))
engagement_input = Input(shape=(1,))
additional_features = Concatenate()([follower_input, engagement_input])
additional_features = Dense(512, activation='relu')(additional_features)
additional_features = Dropout(0.5)(additional_features)

# 모델 병합
combined_input = Concatenate()([image_model.output, content_model.output, hashtags_model.output, additional_features])
combined_output = Dense(1, activation='linear')(combined_input)





58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
# 최종 모델 구성
model = Model(inputs=[image_model.input, content_model.input, hashtags_model.input, follower_input, engagement_input], outputs=combined_output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
# 모델 학습
model.fit([X_image_train, X_content_train, X_hashtags_train, X_follower_train, X_engagement_train], y_train,
          validation_data=([X_image_val, X_content_val, X_hashtags_val, X_follower_val, X_engagement_val], y_val),
          epochs=50, batch_size=256, callbacks=[early_stopping])

# 모델 평가
loss, mae = model.evaluate([X_image_val, X_content_val, X_hashtags_val, X_follower_val, X_engagement_val], y_val)
print(f'Model evaluation - Loss: {loss}, MAE: {mae}')

Epoch 1/50
26/26 [==============================] - 78s 2s/step - loss: 8013999505408.0000 - mae: 1862828.0000 - val_loss: 2334005657600.0000 - val_mae: 1039580.7500
Epoch 2/50
26/26 [==============================] - 16s 606ms/step - loss: 5456704569344.0000 - mae: 1555753.8750 - val_loss: 2338634072064.0000 - val_mae: 1074305.1250
Epoch 3/50
26/26 [==============================] - 14s 552ms/step - loss: 4212325351424.0000 - mae: 1369367.5000 - val_loss: 2309193728000.0000 - val_mae: 1057732.5000
Epoch 4/50
26/26 [==============================] - 14s 538ms/step - loss: 3617058193408.0000 - mae: 1279204.0000 - val_loss: 2324746731520.0000 - val_mae: 1067370.7500
Epoch 5/50
26/26 [==============================] - 14s 524ms/step - loss: 3222485860352.0000 - mae: 1217816.7500 - val_loss: 2496822771712.0000 - val_mae: 1123403.5000
Epoch 6/50
26/26 [==============================] - 13s 513ms/step - loss: 2999496736768.0000 - mae: 1178155.6250 - val_loss: 2307775004672.0000 - val_mae: 10

##lstm, 어텐션 모델(텍스트만)

In [ ]:
from tensorflow.keras.layers import Attention, Concatenate
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Attention, Concatenate, Input, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.models import Model


# 텍스트 입력 모델 (Content)
content_input = Input(shape=(max_seq_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_seq_length)(content_input)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)  # LSTM 층 추가
attention_output = Attention()([lstm_layer, lstm_layer])  # 어텐션 층 추가
lstm_attention_output = Concatenate()([lstm_layer, attention_output])  # LSTM 출력과 어텐션 출력 결합
flatten_layer = Flatten()(lstm_attention_output)
content_output = Dense(512, activation='relu')(flatten_layer)
content_output = Dropout(0.5)(content_output)

# 이미지 입력 모델 (기존 모델 사용)
# 추가 입력 모델 (Follower, Engagement Rate)
follower_input = Input(shape=(1,))
engagement_input = Input(shape=(1,))
additional_features = Concatenate()([follower_input, engagement_input])
additional_features = Dense(512, activation='relu')(additional_features)
additional_features = Dropout(0.5)(additional_features)

# 모델 병합
combined_input = Concatenate()([image_model.output, content_output, hashtags_model.output, additional_features])
combined_output = Dense(1, activation='linear')(combined_input)

# 최종 모델 구성
model_with_attention = Model(inputs=[image_model.input, content_input, hashtags_model.input, follower_input, engagement_input], outputs=combined_output)
model_with_attention.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 학습
model_with_attention.fit([X_image_train, X_content_train, X_hashtags_train, X_follower_train, X_engagement_train], y_train,
                         validation_data=([X_image_val, X_content_val, X_hashtags_val, X_follower_val, X_engagement_val], y_val),
                         epochs=50, batch_size=256)



Epoch 1/50
26/26 [==============================] - 21s 562ms/step - loss: 8992288931840.0000 - mae: 1979210.1250 - val_loss: 2370131460096.0000 - val_mae: 1086423.1250
Epoch 2/50
26/26 [==============================] - 12s 478ms/step - loss: 5768975220736.0000 - mae: 1604931.7500 - val_loss: 2411872124928.0000 - val_mae: 1099824.3750
Epoch 3/50
26/26 [==============================] - 12s 456ms/step - loss: 4535587700736.0000 - mae: 1420244.1250 - val_loss: 2327855759360.0000 - val_mae: 1068986.6250
Epoch 4/50
26/26 [==============================] - 12s 451ms/step - loss: 4103373324288.0000 - mae: 1350673.0000 - val_loss: 2310109921280.0000 - val_mae: 1045977.3750
Epoch 5/50
26/26 [==============================] - 11s 434ms/step - loss: 3705344884736.0000 - mae: 1291083.7500 - val_loss: 2318237433856.0000 - val_mae: 1066109.0000
Epoch 6/50
26/26 [==============================] - 12s 447ms/step - loss: 3394582609920.0000 - mae: 1240947.2500 - val_loss: 2350435008512.0000 - val_mae:

##이미지vgg16, 텍스트 어텐션 병합 모델

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Concatenate, Dense, Embedding, LSTM, Dropout, Flatten
from tensorflow.keras.models import Model

# 이미지 입력 모델
image_input = Input(shape=(224, 224, 3))
base_model = VGG16(weights='imagenet', include_top=False)
image_features = base_model(image_input)
image_features = GlobalAveragePooling2D()(image_features)

# 텍스트 입력 모델 (Content)
content_input = Input(shape=(max_seq_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_seq_length)(content_input)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
attention_output = Attention()([lstm_layer, lstm_layer])
lstm_attention_output = Concatenate()([lstm_layer, attention_output])
flatten_layer = Flatten()(lstm_attention_output)
content_output = Dense(512, activation='relu')(flatten_layer)
content_output = Dropout(0.5)(content_output)

# 해시태그 입력 모델 (Hashtags)
hashtags_input = Input(shape=(max_seq_length,))
hashtags_embedding_layer = Embedding(vocab_size, 100, input_length=max_seq_length)(hashtags_input)
hashtags_lstm_layer = LSTM(128, return_sequences=True)(hashtags_embedding_layer)
hashtags_attention_output = Attention()([hashtags_lstm_layer, hashtags_lstm_layer])
hashtags_lstm_attention_output = Concatenate()([hashtags_lstm_layer, hashtags_attention_output])
hashtags_flatten_layer = Flatten()(hashtags_lstm_attention_output)
hashtags_output = Dense(512, activation='relu')(hashtags_flatten_layer)
hashtags_output = Dropout(0.5)(hashtags_output)

# 추가 입력 모델 (Follower, Engagement Rate)
follower_input = Input(shape=(1,))
engagement_input = Input(shape=(1,))
additional_features = Concatenate()([follower_input, engagement_input])
additional_features = Dense(512, activation='relu')(additional_features)
additional_features = Dropout(0.5)(additional_features)

# 모델 병합
combined_input = Concatenate()([image_features, content_output, hashtags_output, additional_features])
combined_output = Dense(1, activation='linear')(combined_input)

# 최종 모델 구성
model_with_attention_and_image = Model(inputs=[image_input, content_input, hashtags_input, follower_input, engagement_input], outputs=combined_output)
model_with_attention_and_image.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 학습
model_with_attention_and_image.fit([X_image_train, X_content_train, X_hashtags_train, X_follower_train, X_engagement_train], y_train,
                                   validation_data=([X_image_val, X_content_val, X_hashtags_val, X_follower_val, X_engagement_val], y_val),
                                   epochs=50, batch_size=256)



Epoch 1/50
26/26 [==============================] - 22s 570ms/step - loss: 17693392502784.0000 - mae: 2315584.7500 - val_loss: 2457662652416.0000 - val_mae: 1112919.8750
Epoch 2/50
26/26 [==============================] - 13s 505ms/step - loss: 5363225591808.0000 - mae: 1553207.1250 - val_loss: 2343330643968.0000 - val_mae: 1076139.1250
Epoch 3/50
26/26 [==============================] - 13s 503ms/step - loss: 5067030659072.0000 - mae: 1494583.5000 - val_loss: 2331922661376.0000 - val_mae: 1037770.0625
Epoch 4/50
26/26 [==============================] - 13s 503ms/step - loss: 4567873355776.0000 - mae: 1427442.2500 - val_loss: 2293384871936.0000 - val_mae: 1033869.4375
Epoch 5/50
26/26 [==============================] - 13s 502ms/step - loss: 4095465226240.0000 - mae: 1348793.8750 - val_loss: 2325876572160.0000 - val_mae: 1070543.0000
Epoch 6/50
26/26 [==============================] - 13s 504ms/step - loss: 3701033402368.0000 - mae: 1282128.0000 - val_loss: 2283894996992.0000 - val_mae

## 팔로워, 참여도 추가 버전(텍스트, 수치, 이미지 데이터별로)

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Content'].tolist() + data['Hashtags'].tolist())
max_seq_length = max([len(seq) for seq in tokenizer.texts_to_sequences(data['Content'].tolist() + data['Hashtags'].tolist())])
vocab_size = len(tokenizer.word_index) + 1

def preprocess_text(texts, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

X_text = preprocess_text(data['Content'].fillna('') + ' ' + data['Hashtags'].fillna(''), tokenizer, max_seq_length)

# 수치형 데이터 (Engagement_Rate, Follower)
X_numeric = np.vstack((data['Engagement_Rate'].values, data['Follower'].values)).T

# 데이터 준비
X_image = all_image_data
y = data['Likes'].values

# 2. 데이터 분할
X_text_train, X_text_val, X_image_train, X_image_val, X_numeric_train, X_numeric_val, y_train, y_val = train_test_split(
    X_text, X_image, X_numeric, y, test_size=0.2, random_state=42)

# 3. 모델 구성
# 이미지 입력 모델
image_input = Input(shape=(224, 224, 3))
base_model = VGG16(weights='imagenet', include_top=False)
image_features = base_model(image_input)
image_features = Flatten()(image_features)
image_features = Dense(512, activation='relu')(image_features)
image_features = Dropout(0.5)(image_features)

# 텍스트 입력 모델
text_input = Input(shape=(max_seq_length,))
text_embedding = Embedding(vocab_size, 100, input_length=max_seq_length)(text_input)
text_features = Flatten()(text_embedding)
text_features = Dense(512, activation='relu')(text_features)
text_features = Dropout(0.5)(text_features)

# 수치형 데이터 입력 모델 (Engagement_Rate, Follower)
numeric_input = Input(shape=(2,))
numeric_features = Dense(32, activation='relu')(numeric_input)
numeric_features = Dropout(0.5)(numeric_features)

# 모델 병합
merged = Concatenate()([image_features, text_features, numeric_features])
output = Dense(1, activation='linear')(merged)

# 모델 구성
model = Model(inputs=[image_input, text_input, numeric_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])




In [ ]:
# 4. 모델 학습
model.fit(
    [X_image_train, X_text_train, X_numeric_train], y_train,
    epochs=50, batch_size=256,
    validation_data=([X_image_val, X_text_val, X_numeric_val], y_val)
)

# 5. 모델 평가
loss, mae = model.evaluate([X_image_val, X_text_val, X_numeric_val], y_val)
print(f"Validation Loss: {loss}, Validation MAE: {mae}")

Epoch 1/50
26/26 [==============================] - 20s 585ms/step - loss: 632806858293248.0000 - mae: 7496817.5000 - val_loss: 9559551770624.0000 - val_mae: 2279052.5000
Epoch 2/50
26/26 [==============================] - 14s 530ms/step - loss: 14245702926336.0000 - mae: 2606860.2500 - val_loss: 3629786333184.0000 - val_mae: 1455224.2500
Epoch 3/50
26/26 [==============================] - 13s 503ms/step - loss: 9691236139008.0000 - mae: 2181070.0000 - val_loss: 3118833074176.0000 - val_mae: 1357159.3750
Epoch 4/50
26/26 [==============================] - 12s 482ms/step - loss: 8866597175296.0000 - mae: 2110858.2500 - val_loss: 2809160269824.0000 - val_mae: 1166887.1250
Epoch 5/50
26/26 [==============================] - 12s 480ms/step - loss: 8313990807552.0000 - mae: 2028620.8750 - val_loss: 2642789269504.0000 - val_mae: 1234822.6250
Epoch 6/50
26/26 [==============================] - 13s 481ms/step - loss: 7288846811136.0000 - mae: 1850619.0000 - val_loss: 2500415979520.0000 - val_m

## 팔로워, 참여도 없는 버전 (게시글, 해시태그 합침)

In [ ]:


# 이미지 데이터 전처리
def preprocess_images(folders):
    all_image_data = []
    for folder in folders:
        image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder) if filename.endswith(('.png', '.jpg', '.jpeg'))]
        for img_path in image_paths:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            all_image_data.append(img_array)
    return np.vstack(all_image_data)

folders = [
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/윈터',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/카리나',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/지젤/지젤',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/닝닝/에스파_닝닝'
]
all_image_data = preprocess_images(folders)

# 텍스트 데이터 임베딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Content'].tolist() + data['Hashtags'].tolist())
max_seq_length = max([len(seq) for seq in tokenizer.texts_to_sequences(data['Content'].tolist() + data['Hashtags'].tolist())])
vocab_size = len(tokenizer.word_index) + 1

def preprocess_text(texts, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

X_text = preprocess_text(data['Content'] + ' ' + data['Hashtags'], tokenizer, max_seq_length)

# 데이터 준비
X_image = all_image_data
y = data['Likes'].values

# 데이터 분할
X_text_train, X_text_val, X_image_train, X_image_val, y_train, y_val = train_test_split(X_text, X_image, y, test_size=0.2, random_state=42)

# 이미지 입력 모델
image_input = Input(shape=(224, 224, 3))
base_model = VGG16(weights='imagenet', include_top=False)
image_features = base_model(image_input)
image_features = Flatten()(image_features)
image_features = Dense(512, activation='relu')(image_features)
image_features = Dropout(0.5)(image_features)

# 텍스트 입력 모델
text_input = Input(shape=(max_seq_length,))
text_embedding = Embedding(vocab_size, 100, input_length=max_seq_length)(text_input)
text_features = Flatten()(text_embedding)
text_features = Dense(512, activation='relu')(text_features)
text_features = Dropout(0.5)(text_features)

# 모델 병합
merged = Concatenate()([image_features, text_features])
output = Dense(1, activation='linear')(merged)

# 모델 구성
model = Model(inputs=[image_input, text_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])



In [ ]:
# 모델 학습
model.fit([X_image_train, X_text_train], y_train, validation_data=([X_image_val, X_text_val], y_val), epochs=50, batch_size=32)

# 모델 평가
loss, mae = model.evaluate([X_image_val, X_text_val], y_val)
print(f'Model evaluation - Loss: {loss}, MAE: {mae}')

Epoch 1/50
12/12 [==============================] - 5s 452ms/step - loss: 746243031040.0000 - mae: 660891.0000 - val_loss: 1098024419328.0000 - val_mae: 708864.5000
Epoch 2/50
12/12 [==============================] - 5s 447ms/step - loss: 731298791424.0000 - mae: 647103.3750 - val_loss: 1369904250880.0000 - val_mae: 805981.1875
Epoch 3/50
12/12 [==============================] - 5s 410ms/step - loss: 880840736768.0000 - mae: 715773.3750 - val_loss: 961953529856.0000 - val_mae: 715656.1875
Epoch 4/50
12/12 [==============================] - 5s 427ms/step - loss: 755281166336.0000 - mae: 668967.8125 - val_loss: 1288878424064.0000 - val_mae: 774724.6250
Epoch 5/50
12/12 [==============================] - 5s 429ms/step - loss: 786385797120.0000 - mae: 674233.7500 - val_loss: 953154404352.0000 - val_mae: 736973.9375
Epoch 6/50
12/12 [==============================] - 5s 440ms/step - loss: 751526674432.0000 - mae: 669474.1875 - val_loss: 1017481854976.0000 - val_mae: 694468.1875
Epoch 7/50
1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 데이터 분할
X = data[['Follower', 'Engagement_Rate']]  # 수치형 특성
y = data['Likes']  # 타겟 변수
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 검증 세트에 대한 예측
y_pred = rf_model.predict(X_val)

# 성능 평가
mae = mean_absolute_error(y_val, y_pred)
print(f'랜덤 포레스트 모델의 평균 절대 오차: {mae}')


랜덤 포레스트 모델의 평균 절대 오차: 6951.716554673183


## 팔로워, 참여도 없는 버전 (게시글, 해시태그 따로따로)

In [ ]:
import re
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/에스파.csv')  # 데이터 파일 경로를 적절히 수정해주세요.

# 텍스트 데이터 전처리
data['Content'] = data['Content'].fillna('')
data['Hashtags'] = data['Hashtags'].fillna('')
data['Likes'] = data['Likes'].astype(str)

def preprocess_likes(likes):
    if '좋아합니다' in likes:
        match = re.search(r'외\s+(\d+(\.\d+)?)만', likes)
        if match:
            likes_value = float(match.group(1)) * 10000
            return likes_value
    elif '만개' in likes:
        likes_value = float(likes.replace('만개', '')) * 10000
        return likes_value
    return float(likes)

data['Likes'] = data['Likes'].apply(preprocess_likes)

# 이미지 데이터 전처리
def preprocess_images(folders):
    all_image_data = []
    for folder in folders:
        image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder) if filename.endswith(('.png', '.jpg', '.jpeg'))]
        for img_path in image_paths:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            all_image_data.append(img_array)
    return np.vstack(all_image_data)

folders = [
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/윈터',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/카리나',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/지젤/지젤',
    '/content/drive/MyDrive/Colab Notebooks/kdt/크롤링/닝닝/에스파_닝닝'
]
all_image_data = preprocess_images(folders)

# 텍스트 데이터 임베딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Content'].tolist() + data['Hashtags'].tolist())
max_seq_length = max([len(seq) for seq in tokenizer.texts_to_sequences(data['Content'].tolist() + data['Hashtags'].tolist())])
vocab_size = len(tokenizer.word_index) + 1

def preprocess_text(texts, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

X_content = preprocess_text(data['Content'], tokenizer, max_seq_length)
X_hashtags = preprocess_text(data['Hashtags'], tokenizer, max_seq_length)

# 데이터 준비
X_image = all_image_data
y = data['Likes'].values

# 데이터 분할
X_content_train, X_content_val, X_hashtags_train, X_hashtags_val, X_image_train, X_image_val, y_train, y_val = train_test_split(
    X_content, X_hashtags, X_image, y, test_size=0.2, random_state=42)

# 이미지 입력 모델
image_input = Input(shape=(224, 224, 3))
base_model = VGG16(weights='imagenet', include_top=False)
image_features = base_model(image_input)
image_features = Flatten()(image_features)
image_features = Dense(512, activation='relu')(image_features)
image_features = Dropout(0.5)(image_features)

# 텍스트 입력 모델 (Content)
content_input = Input(shape=(max_seq_length,))
content_embedding = Embedding(vocab_size, 100, input_length=max_seq_length)(content_input)
content_features = Flatten()(content_embedding)
content_features = Dense(512, activation='relu')(content_features)
content_features = Dropout(0.5)(content_features)

# 텍스트 입력 모델 (Hashtags)
hashtags_input = Input(shape=(max_seq_length,))
hashtags_embedding = Embedding(vocab_size, 100, input_length=max_seq_length)(hashtags_input)
hashtags_features = Flatten()(hashtags_embedding)
hashtags_features = Dense(512, activation='relu')(hashtags_features)
hashtags_features = Dropout(0.5)(hashtags_features)

# 모델 병합
merged = Concatenate()([image_features, content_features, hashtags_features])
output = Dense(1, activation='linear')(merged)

# 모델 구성
model = Model(inputs=[image_input, content_input, hashtags_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# 모델 학습
model.fit([X_image_train, X_content_train, X_hashtags_train], y_train,
          validation_data=([X_image_val, X_content_val, X_hashtags_val], y_val),
          epochs=50, batch_size=32, callbacks=[early_stopping])

# 모델 평가
loss, mae = model.evaluate([X_image_val, X_content_val, X_hashtags_val], y_val)
print(f'Model evaluation - Loss: {loss}, MAE: {mae}')


58889256/58889256 [==============================] - 4s 0us/step
Epoch 1/50
12/12 [==============================] - 51s 2s/step - loss: 4321807171584.0000 - mae: 1662879.8750 - val_loss: 3936393101312.0000 - val_mae: 1734458.3750
Epoch 2/50
12/12 [==============================] - 6s 496ms/step - loss: 3212336168960.0000 - mae: 1605253.7500 - val_loss: 3440018456576.0000 - val_mae: 1586437.0000
Epoch 3/50
12/12 [==============================] - 6s 508ms/step - loss: 2286886322176.0000 - mae: 1280807.8750 - val_loss: 1397567389696.0000 - val_mae: 824013.3125
Epoch 4/50
12/12 [==============================] - 5s 438ms/step - loss: 1505508982784.0000 - mae: 959836.6875 - val_loss: 1625215467520.0000 - val_mae: 1096320.7500
Epoch 5/50
12/12 [==============================] - 6s 474ms/step - loss: 1729490059264.0000 - mae: 1042975.1875 - val_loss: 1412916314112.0000 - val_mae: 1010106.5625
Epoch 6/50
12/12 [==============================] - 6s 471ms/step - loss: 1030089211904.0000 - mae: